# Incluidas as bibliotecas

In [1]:
import pandas as pd

# Definidas as planilhas

In [2]:
# Acionamentos
df = pd.read_excel('Acionamentos.xlsx')
# Convertendo a coluna de data e extraindo apenas a parte da data
df['Data Inclusão'] = pd.to_datetime(df['Data Inclusão']).dt.date
# Operadores
df2 = pd.read_excel('Operadores.xlsx')
# Ocorrências
df3 = pd.read_excel('Ocorrencias.xlsx')

df_main = df[df['Cobrador'].isin(df2['Cobrador'])]

C:\Users\davi.melo\AppData\Local\Temp\ipykernel_30580\3190163021.py:4: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Data Inclusão'] = pd.to_datetime(df['Data Inclusão']).dt.date


## EDA

In [3]:
df_main.head(5)

,fase,Atraso,Cobrador,Data Inclusão,Duração,Horário,Acionamento,Situação de Cobrança,Origem,Contratante,...,Produto,Arquivo,Fila,empresa,Sub Classificação Ocorrência,DE-PARA,Data de Exportacao,Resultado do envio ao WebService,Observacao,assessoria
0,NaN,NaN,Vanessa Silva,2024-09-30,NaN,NaN,Não Atende,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
1,NaN,NaN,Amanda Lais,2024-09-30,NaN,NaN,Não Atende,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
2,NaN,NaN,Mariana Nascimento,2024-09-30,NaN,NaN,Recado Caixa Postal,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
3,NaN,NaN,Mariana Nascimento,2024-09-30,NaN,NaN,Desconhece Divida,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
4,NaN,NaN,Amanda Lais,2024-09-30,NaN,NaN,Mensagem de Operadora,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN


In [4]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52691 entries, 0 to 58137
Data columns (total 33 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   fase                              42057 non-null  object        
 1   Atraso                            0 non-null      float64       
 2   Cobrador                          52691 non-null  object        
 3   Data Inclusão                     52691 non-null  object        
 4   Duração                           44590 non-null  object        
 5   Horário                           44235 non-null  object        
 6   Acionamento                       52691 non-null  object        
 7   Situação de Cobrança              44551 non-null  object        
 8   Origem                            44590 non-null  object        
 9   Contratante                       44590 non-null  object        
 10  ID                                52691 non-null  f

## Definida planilha de exibição

In [5]:
acionamentos_operador = df_main.groupby(['Cobrador', 'Data Inclusão']).agg({
    'ID': 'nunique',
}).rename(columns={'ID': 'Clientes alcançados'}).reset_index()

## Coluna de tentativas sendo a contagem de acionamentos por id de cliente

In [6]:
acionamentos_operador['Tentativas'] = df_main.groupby(['Cobrador', 'Data Inclusão'])['ID'].count().values

## Realizando um merge para poder atribuir as carteiras e supervisores dos operadores

In [7]:
acionamentos_operador = pd.merge(acionamentos_operador, df2[['Cobrador', 'Carteira']], on='Cobrador', how='left')
acionamentos_operador = pd.merge(acionamentos_operador, df2[['Cobrador', 'Supervisor']], on='Cobrador', how='left')

## Realizando o merge entre as tabelas de ocorrências e acionamentos

In [8]:
df_merged = pd.merge(df_main, df3[['Ocorrência', 'Classificação']], 
                     left_on='Acionamento', right_on='Ocorrência', how='left')

KeyError: "['Ocorrência'] not in index"

In [24]:
df_merged.head(3)

,fase,Atraso,Cobrador,Data Inclusão,Duração,Horário,Acionamento,Situação de Cobrança,Origem,Contratante,...,Fila,empresa,Sub Classificação Ocorrência,DE-PARA,Data de Exportacao,Resultado do envio ao WebService,Observacao,assessoria,Ocorrência,Classificação
0,SEM FASE DEFINIDA,NaN,Ana Beatriz,2024-09-19,00:01:00,13:00,ABANDONO DE LIGAÇÃO,CONTATO FEITO POR ASSESSORIA DE COBRANÇA,FILA,LOJAS CATTAN,...,17015 -GCD_CPP E CPC_,NaN,NaN,8.333063e+09,2024-09-19 13:34:46.287,WS - Acionamento incluso com sucesso,NaN,NaN,ABANDONO DE LIGAÇÃO,ALÔ
1,SEM FASE DEFINIDA,NaN,Ana Beatriz,2024-09-19,00:01:11,13:00,ABANDONO DE LIGAÇÃO,CONTATO FEITO POR ASSESSORIA DE COBRANÇA,FILA,LOJAS CATTAN,...,17015 -GCD_CPP E CPC_,NaN,NaN,8.333063e+09,2024-09-19 13:40:25.823,WS - Acionamento incluso com sucesso,NaN,NaN,ABANDONO DE LIGAÇÃO,ALÔ
2,SEM FASE DEFINIDA,NaN,Ana Beatriz,2024-09-19,00:01:18,14:00,ABANDONO DE LIGAÇÃO,CONTATO FEITO POR ASSESSORIA DE COBRANÇA,FILA,LOJAS CATTAN,...,17015 -GCD_CPP E CPC_,NaN,NaN,8.333063e+09,2024-09-19 14:04:20.407,WS - Acionamento incluso com sucesso,NaN,NaN,ABANDONO DE LIGAÇÃO,ALÔ


## Contagem unique de ALÔ, CPC e CPC TARGET por 'Cobrador' e 'Data Inclusão'


In [29]:
contagem_alo = df_merged[df_merged['Classificação'].isin(['ALÔ', 'CPC', 'CPC - TARGET'])].groupby(['Cobrador', 'Data Inclusão'])['ID'].nunique().reset_index()
contagem_cpc = df_merged[df_merged['Classificação'].isin(['CPC', 'CPC - TARGET'])].groupby(['Cobrador', 'Data Inclusão'])['ID'].nunique().reset_index()
contagem_cpcT = df_merged[df_merged['Classificação'].isin(['CPC - TARGET'])].groupby(['Cobrador', 'Data Inclusão'])['ID'].nunique().reset_index()
contagem_acordo = df_merged[df_merged['Classificação'].isin(['ACORDO'])].groupby(['Cobrador', 'Data Inclusão'])['ID'].nunique().reset_index()
contagem_acordo

,Cobrador,Data Inclusão,ID
0,ANDREA NASCIMENTO,2024-09-05,1
1,ANDREA NASCIMENTO,2024-09-10,1
2,ANDREA NASCIMENTO,2024-09-11,3
3,ANDREA NASCIMENTO,2024-09-13,2
4,ANDREA NASCIMENTO,2024-09-16,1
...,...,...,...
275,YASMIM CRUZ,2024-09-18,1
276,YASMIM CRUZ,2024-09-19,5
277,YASMIM CRUZ,2024-09-20,1
278,YASMIM CRUZ,2024-09-27,1


## Renomeando as colunas


In [36]:
contagem_alo.rename(columns={'ID': 'ALÔ'}, inplace=True)
contagem_cpc.rename(columns={'ID': 'CPC'}, inplace=True)
contagem_cpcT.rename(columns={'ID': 'CPC - TARGET'}, inplace=True)
contagem_acordo.rename(columns={'ID': 'ACORDOS'}, inplace=True)

## Definindo as colunas de ALÔ, CPC e CPC - TARGET

In [39]:
acionamentos_operador = acionamentos_operador.merge(contagem_alo, on=['Cobrador', 'Data Inclusão'], how='left')
acionamentos_operador['%ALÔ'] = ((acionamentos_operador['ALÔ'] / acionamentos_operador['Clientes alcançados']) * 100).round(1)

acionamentos_operador = acionamentos_operador.merge(contagem_cpc, on=['Cobrador', 'Data Inclusão'], how='left')
acionamentos_operador['%CPC'] = ((acionamentos_operador['CPC'] / acionamentos_operador['ALÔ']) * 100).round(1)

acionamentos_operador = acionamentos_operador.merge(contagem_cpcT, on=['Cobrador', 'Data Inclusão'], how='left')
acionamentos_operador['%TARGET'] = ((acionamentos_operador['CPC - TARGET'] / acionamentos_operador['CPC']) * 100).round(1)

acionamentos_operador = acionamentos_operador.merge(contagem_acordo, on=['Cobrador', 'Data Inclusão'], how='left')
acionamentos_operador['%CONVERSÃO'] = ((acionamentos_operador['ACORDOS'] / acionamentos_operador['CPC - TARGET']) * 100).round(1)

In [36]:
acionamentos_operador

,Cobrador,Data Inclusão,Clientes alcançados,Tentativas,Carteira,Supervisor,ALÔ,%ALÔ,CPC,%CPC,CPC - TARGET,%TARGET
0,ANDREA NASCIMENTO,2024-09-02,46,48,DIVERSOS,Alyne Barbosa,29.0,63.0,9.0,31.0,9.0,100.0
1,ANDREA NASCIMENTO,2024-09-03,65,68,DIVERSOS,Alyne Barbosa,31.0,47.7,12.0,38.7,11.0,91.7
2,ANDREA NASCIMENTO,2024-09-04,75,76,DIVERSOS,Alyne Barbosa,12.0,16.0,5.0,41.7,3.0,60.0
3,ANDREA NASCIMENTO,2024-09-05,29,34,DIVERSOS,Alyne Barbosa,16.0,55.2,11.0,68.8,9.0,81.8
4,ANDREA NASCIMENTO,2024-09-06,26,29,DIVERSOS,Alyne Barbosa,13.0,50.0,7.0,53.8,7.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...
505,YASMIM CRUZ,2024-09-19,55,58,CONDOMINIO,Alyne Barbosa,14.0,25.5,14.0,100.0,14.0,100.0
506,YASMIM CRUZ,2024-09-20,48,48,CONDOMINIO,Alyne Barbosa,15.0,31.2,14.0,93.3,12.0,85.7
507,YASMIM CRUZ,2024-09-23,68,70,CONDOMINIO,Alyne Barbosa,20.0,29.4,11.0,55.0,5.0,45.5
508,YASMIM CRUZ,2024-09-27,97,101,CONDOMINIO,Alyne Barbosa,9.0,9.3,9.0,100.0,8.0,88.9


## Reordenando as colunas pela lista

In [38]:
Ordem = [
    'Cobrador',
    'Carteira', 
    'Supervisor', 
    'Data Inclusão', 
    'Tentativas', 
    'Clientes alcançados', 
    'ALÔ', 
    '%ALÔ', 
    'CPC', 
    '%CPC', 
    'CPC - TARGET', 
    '%TARGET',
    'ACORDOS',
    '%CONVERSÃO' 
]

In [37]:
acionamentos_visualizacao = acionamentos_operador.reindex(columns=Ordem)
acionamentos_visualizacao

,Cobrador,Carteira,Supervisor,Data Inclusão,Tentativas,Clientes alcançados,ALÔ,%ALÔ,CPC,%CPC,CPC - TARGET,%TARGET
0,ANDREA NASCIMENTO,DIVERSOS,Alyne Barbosa,2024-09-02,48,46,29.0,63.0,9.0,31.0,9.0,100.0
1,ANDREA NASCIMENTO,DIVERSOS,Alyne Barbosa,2024-09-03,68,65,31.0,47.7,12.0,38.7,11.0,91.7
2,ANDREA NASCIMENTO,DIVERSOS,Alyne Barbosa,2024-09-04,76,75,12.0,16.0,5.0,41.7,3.0,60.0
3,ANDREA NASCIMENTO,DIVERSOS,Alyne Barbosa,2024-09-05,34,29,16.0,55.2,11.0,68.8,10.0,90.9
4,ANDREA NASCIMENTO,DIVERSOS,Alyne Barbosa,2024-09-06,29,26,13.0,50.0,7.0,53.8,7.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...
505,YASMIM CRUZ,CONDOMINIO,Alyne Barbosa,2024-09-19,58,55,19.0,34.5,19.0,100.0,19.0,100.0
506,YASMIM CRUZ,CONDOMINIO,Alyne Barbosa,2024-09-20,48,48,16.0,33.3,15.0,93.8,13.0,86.7
507,YASMIM CRUZ,CONDOMINIO,Alyne Barbosa,2024-09-23,70,68,20.0,29.4,11.0,55.0,5.0,45.5
508,YASMIM CRUZ,CONDOMINIO,Alyne Barbosa,2024-09-27,101,97,9.0,9.3,9.0,100.0,8.0,88.9
